# Advanced SQL Tutorial

In [2]:
#Step 1: Setup SQLite in Colab
import sqlite3
import pandas as pd

In [3]:
#Step 2: Create a connection and cursor
# Create an in-memory database
conn = sqlite3.connect(':memory:')  # creates a new SQLite database connection.  # ':memory:', it tells SQLite to create a temporary database in RAM (memory).
cursor = conn.cursor()  #  creates a cursor object from the database connection #  cursor object, which is used to interact with the database.

In [4]:
cursor.execute('''
CREATE TABLE Employees (
    emp_id INT PRIMARY KEY,
    name VARCHAR(50),
    department VARCHAR(50),
    salary INT,
    gender CHAR(1),
    state VARCHAR(30),
    join_date DATE,
    rating DECIMAL(4,2)
);
''')

## Insert Sample data

In [5]:
employees_data = [
    (1, 'Asha', 'HR', 60000, 'F', 'Delhi', '2024-01-15', 4.8),
    (2, 'Ramesh', 'IT', 72000, 'M', 'Punjab', '2024-02-20', 4.2),
    (3, 'Priya', 'Finance', 68000, 'F', 'Delhi', '2024-01-15', 4.6),
    (4, 'Kabir', 'IT', 70000, 'M', 'Tamil Nadu', '2024-02-20', 4.7),
    (5, 'Alok', 'HR', 62000, 'M', 'Gujarat', '2024-03-01', 4.3),
    (6, 'Sneha', 'Marketing', 64000, 'F', 'Kerala', '2024-03-01', 4.5),
    (7, 'Ritika', 'Finance', 69000, 'F', 'Maharashtra', '2024-02-20', 4.9),
    (8, 'Manoj', 'IT', 71000, 'M', 'Delhi', '2024-03-01', 4.1),
    (9, 'Deepa', 'HR', 63000, 'F', 'Punjab', '2024-01-15', 4.4),
    (10, 'Amit', 'Marketing', 67000, 'M', 'Kerala', '2024-02-20', 4.0)
]

cursor.executemany('''
INSERT INTO Employees (emp_id, name, department, salary, gender, state, join_date, rating)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', employees_data)

conn.commit()


## Display the table

In [6]:
# Display all rows from Employees table
cursor.execute("SELECT * FROM Employees")
rows = cursor.fetchall()

# Print header
print("emp_id | name   | department | salary | gender | state      | join_date  | rating")
print("-" * 80)
# Print each row
for row in rows:
    print("{:<7} {:<8} {:<11} {:<7} {:<7} {:<11} {:<11} {:<6}".format(*row))


emp_id | name   | department | salary | gender | state      | join_date  | rating
--------------------------------------------------------------------------------
1       Asha     HR          60000   F       Delhi       2024-01-15  4.8   
2       Ramesh   IT          72000   M       Punjab      2024-02-20  4.2   
3       Priya    Finance     68000   F       Delhi       2024-01-15  4.6   
4       Kabir    IT          70000   M       Tamil Nadu  2024-02-20  4.7   
5       Alok     HR          62000   M       Gujarat     2024-03-01  4.3   
6       Sneha    Marketing   64000   F       Kerala      2024-03-01  4.5   
7       Ritika   Finance     69000   F       Maharashtra 2024-02-20  4.9   
8       Manoj    IT          71000   M       Delhi       2024-03-01  4.1   
9       Deepa    HR          63000   F       Punjab      2024-01-15  4.4   
10      Amit     Marketing   67000   M       Kerala      2024-02-20  4     


## Advanced Query Examples

## Example 1. Top 3 Pairs with Highest Salary Difference (Excluding Same Employees)

In [7]:
cursor.execute('''
SELECT A.name AS emp_A, B.name AS emp_B,
       ABS(A.salary - B.salary) AS salary_diff
FROM Employees A, Employees B
WHERE A.emp_id < B.emp_id
ORDER BY salary_diff DESC
LIMIT 3;
''')

# A.name:Refers to the name column of the first employee in the pair.
# B.name: Refers to the name column of the second employee in the pair.
# A.emp_id < B.emp_id - why to avoid: Duplicate pairs (like Ramesh–Asha and Asha–Ramesh), Self-pairs (like Asha–Asha)

In [8]:
# fetch and display the result
results = cursor.fetchall()

# Print nicely
print("Top 3 Pairs with Highest Salary Difference:")
print("emp_A     | emp_B     | salary_diff")
print("-------------------------------------")
for row in results:
    print("{:<10} {:<10} {:<}".format(row[0], row[1], row[2]))



Top 3 Pairs with Highest Salary Difference:
emp_A     | emp_B     | salary_diff
-------------------------------------
Asha       Ramesh     12000
Asha       Manoj      11000
Asha       Kabir      10000


## Example 2 : Employees with Same Join Date and Rating Difference ≤ 0.5

In [12]:
# Find employee pairs who: Have the same join date,  Are different people, Have a rating difference ≤ 0.5

cursor.execute('''
SELECT A.name AS emp1, B.name AS emp2, A.join_date,
       A.rating AS rating1, B.rating AS rating2,
       ABS(A.rating - B.rating) AS diff
FROM Employees A, Employees B
WHERE A.emp_id < B.emp_id
  AND A.join_date = B.join_date
  AND ABS(A.rating - B.rating) <= 0.5;
''')

# A.name and B.name refer to two different employees with the same join date.

# ABS(A.rating - B.rating) calculates the absolute rating difference.

# A.emp_id < B.emp_id avoids duplicate and self-pairs.

In [13]:
results = cursor.fetchall()

# Print header
print("emp1     | emp2     | join_date  | rating1 | rating2 | diff")
print("---------------------------------------------------------------")

# Print each row
for row in results:
    print("{:<9} {:<9} {:<11} {:<8} {:<8} {:<}".format(*row))


emp1     | emp2     | join_date  | rating1 | rating2 | diff
---------------------------------------------------------------
Asha      Priya     2024-01-15  4.8      4.6      0.20000000000000018
Asha      Deepa     2024-01-15  4.8      4.4      0.39999999999999947
Ramesh    Kabir     2024-02-20  4.2      4.7      0.5
Ramesh    Amit      2024-02-20  4.2      4        0.20000000000000018
Priya     Deepa     2024-01-15  4.6      4.4      0.1999999999999993
Kabir     Ritika    2024-02-20  4.7      4.9      0.20000000000000018
Alok      Sneha     2024-03-01  4.3      4.5      0.20000000000000018
Alok      Manoj     2024-03-01  4.3      4.1      0.20000000000000018
Sneha     Manoj     2024-03-01  4.5      4.1      0.40000000000000036


## Example 3 :

In [14]:
# Step 1: Get Average Salary per Department into a temp table

cursor.execute('''
CREATE TEMP VIEW dept_avg_salary AS
SELECT department, AVG(salary) AS avg_salary
FROM Employees
GROUP BY department;
''')

In [15]:
# Step 2: Do a self-join on the temp view to compare departments

cursor.execute('''
SELECT A.department AS dept_A, B.department AS dept_B,
       ABS(A.avg_salary - B.avg_salary) AS avg_salary_diff
FROM dept_avg_salary A
JOIN dept_avg_salary B
  ON A.department < B.department
ORDER BY avg_salary_diff DESC;
''')

In [16]:
results = cursor.fetchall()
print("dept_A       | dept_B       | avg_salary_diff")
print("-----------------------------------------------")
for row in results:
    print("{:<12} {:<12} {:<.2f}".format(*row))


dept_A       | dept_B       | avg_salary_diff
-----------------------------------------------
HR           IT           9333.33
Finance      HR           6833.33
IT           Marketing    5500.00
HR           Marketing    3833.33
Finance      Marketing    3000.00
Finance      IT           2500.00


## Example 4 :

In [17]:
# Find female-male employee pairs:From different states, Where the male’s rating is higher than the female’s rating

cursor.execute('''
SELECT F.name AS female_name, M.name AS male_name,
       F.state AS female_state, M.state AS male_state,
       F.rating AS female_rating, M.rating AS male_rating
FROM Employees F
JOIN Employees M
  ON F.gender = 'F' AND M.gender = 'M'
WHERE F.state <> M.state
  AND M.rating > F.rating;
''')

# F and M are aliases for the same table (Employees).
# We join female employees (F.gender = 'F') with male employees (M.gender = 'M').
# Only pairs with different states and male rating > female rating are selected.


In [18]:
results = cursor.fetchall()

print("female_name | male_name  | female_state | male_state   | female_rating | male_rating")
print("--------------------------------------------------------------------------------------")
for row in results:
    print("{:<12} {:<11} {:<13} {:<13} {:<14} {:<}".format(*row))


female_name | male_name  | female_state | male_state   | female_rating | male_rating
--------------------------------------------------------------------------------------
Priya        Kabir       Delhi         Tamil Nadu    4.6            4.7
Sneha        Kabir       Kerala        Tamil Nadu    4.5            4.7
Deepa        Kabir       Punjab        Tamil Nadu    4.4            4.7


## Example 5 :

In [21]:
# This query identifies non-transitive (cyclic) rating relationships where: A > B, B > C, and C > A — a logical loop!
cursor.execute('''
SELECT A.name AS emp_A, A.rating AS rating_A,
       B.name AS emp_B, B.rating AS rating_B,
       C.name AS emp_C, C.rating AS rating_C
FROM Employees A
JOIN Employees B ON A.rating > B.rating
JOIN Employees C ON B.rating > C.rating AND C.rating > A.rating;
''')


In [28]:
results = cursor.fetchall()

print("emp_A      | rating_A | emp_B      | rating_B | emp_C      | rating_C")
print("----------------------------------------------------------------------")
for row in results:
    print("{:<5} {:<3} {:<5} {:<3} {:<5} {:<}".format(*row))


emp_A      | rating_A | emp_B      | rating_B | emp_C      | rating_C
----------------------------------------------------------------------


## Example 6 :  Pairs in Same Department and State but Different Gender

In [29]:
cursor.execute('''
SELECT A.name, A.gender, B.name, B.gender, A.department, A.state
FROM Employees A, Employees B
WHERE A.emp_id < B.emp_id
  AND A.department = B.department
  AND A.state = B.state
  AND A.gender <> B.gender;
''')

# Compares all pairs of employees from: The same department, The same state, Different gender
# And skips duplicate/self-pairs using A.emp_id < B.emp_id

In [30]:
results = cursor.fetchall()

print("Name A    | Gender A | Name B    | Gender B | Department  | State")
print("--------------------------------------------------------------------")
for row in results:
    print("{:<10} {:<9} {:<10} {:<9} {:<12} {:<}".format(*row))


Name A    | Gender A | Name B    | Gender B | Department  | State
--------------------------------------------------------------------
Sneha      F         Amit       M         Marketing    Kerala


## Example 7:Department-Wise Max Salary and Who Earns It

In [31]:
# Show each department, its maximum salary, and the employee(s) earning that salary.
cursor.execute('''
SELECT department, name, salary
FROM Employees
WHERE (department, salary) IN (
    SELECT department, MAX(salary)
    FROM Employees
    GROUP BY department
);
''')

# returns each department with its max salary.
# The main query filters only employees matching those department-salary pairs.
# This handles ties too (multiple employees with same max salary in a department).


In [32]:
results = cursor.fetchall()

print("Department   | Name        | Salary")
print("---------------------------------------")
for row in results:
    print("{:<13} {:<11} {:<}".format(*row))


Department   | Name        | Salary
---------------------------------------
IT            Ramesh      72000
Finance       Ritika      69000
HR            Deepa       63000
Marketing     Amit        67000
